### PoC to pull data from the Strava API, and utilize DataBricks lakehouse for storage

In [ ]:
%run ./config.py

In [ ]:
from boto import config
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"


payload = {
    'client_id':  client_id,
    'client_secret': client_secret,
    'code': authorization_code,
    'grant_type': 'authorization_code',
    'f': 'json'
}

print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
refresh_token = res.json()['refresh_token']
access_token = res.json()['access_token']
expire_at_unix_time = res.json()['expires_at']


In [ ]:
#API call to grab all of the acitivites within a personal account
activites_url = "https://www.strava.com/api/v3/athlete/activities"
header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}
my_dataset = requests.get(activites_url, headers=header, params=param).json()

In [ ]:
#get all of the activity ids from the bigger dataset
activity_ids = []
count = 0
while count < len(my_dataset):
    activity_ids.append(my_dataset[count]['id'])
    count += 1 

In [ ]:
#can get the invdividal activites
activity_id_urls = ("{}{}?include_all_efforts= True").format("https://www.strava.com/api/v3/activities/","")
header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}
my_activity_ = requests.get(activity_id_urls, headers=header, params=param).json()

In [ ]:
my_activity_

In [ ]:
len(my_activity_['segment_efforts'])

In [ ]:
#looking at the various segments within the particular activity 
my_activity_['segment_efforts'][15]

In [ ]:
#segment effort API call 
segment_id_url =("{}{}").format( "https://www.strava.com/api/v3/segment_efforts/",) 
header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}
my_segment_ = requests.get(segment_id_url, headers=header, params=param).json()

In [ ]:
my_segment_['segment']['id']
#Some very interesting information in this one, oldest created date
# 'effort_count': 20871,
 #'athlete_count': 1955,
 #'star_count': 11,
segment_id_url =("{}{}").format( "https://www.strava.com/api/v3/segments/",) 
header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}
my_segment = requests.get(segment_id_url, headers=header, params=param).json()

#### Interesting Fields from Json segment 
name': '26th St.',
 'activity_type': 'Ride',
 'distance': 674.65,
  'city': 'San Diego',



   'athlete_segment_stats': {'pr_elapsed_time': 136,
  'pr_date': '2022-04-29',
  'pr_activity_id': 7062090532,
  'effort_count': 18},
 'xoms': {'kom': '45s',
  'qom': '1:24',
  'overall': '45s',


   'effort_count': 20871,
 'athlete_count': 1955,
 'star_count': 11,

#### Helpful sparkcode from databricks cert

from pyspark.sql.functions import * 
#read in 2017 files into dataframe
df_2017 =spark.read.text(batch_2017_path)

#extract columns and appropriate headers
df_new = df_2017.withColumn('submitted_at',  col('value').substr(1,15)) \
.withColumn('order_id', col('value').substr(16,40))\
.withColumn('customer_id', col('value').substr(56,40))\
.withColumn('sales_rep_id', col('value').substr(96,40)) \
.withColumn('sales_rep_ssn', col('value').substr(136,15))\
.withColumn('sales_rep_first_name', col('value').substr(151,15))\
.withColumn('sales_rep_last_name', col('value').substr(166,15))\
.withColumn('sales_rep_address', col('value').substr(181,40))\
.withColumn('sales_rep_city', col('value').substr(221,20))\
.withColumn('sales_rep_state', col('value').substr(241,2))\
.withColumn('sales_rep_zip', col('value').substr(243,5))\
.withColumn('shipping_address_attention', col('value').substr(248,30))\
.withColumn('shipping_address_address', col('value').substr(278,40))\
.withColumn('shipping_address_city', col('value').substr(318,20))\
.withColumn('shipping_address_state', col('value').substr(338,2))\
.withColumn('shipping_address_zip', col('value').substr(340,5))\
.withColumn('product_id', col('value').substr(345,40))\
.withColumn('product_quantity', col('value').substr(385,5))\
.withColumn('product_sold_price', col('value').substr(390,20))\
.drop('value') 


df2 = df_new.select([trim(col(c)).alias(c) for c in df_new.columns]) 
df2= df2.select([when(col(c)== "", None).otherwise(col(c)).alias(c) for c in df2.columns])
df_2 = df2.withColumn("ingest_file_name", input_file_name()) \
        .withColumn("ingested_at", current_timestamp())


df_2.write.format("delta")\
.option("overwriteSchema", "true")\
.mode("overwrite")\
.save(batch_target_path)